In [1]:
# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import glob
import re

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline

print('Libraries imported')

Libraries imported


In [2]:
path = r'C:\Metis_Bootcamp\Regression_project\data\joined-data' # data path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    data = pd.read_csv(filename, index_col=0, header=0)
    li.append(data)

df = pd.concat(li, axis=0, ignore_index=True)

df

,Name,Mileage,Address,Rating,Fuel Type,City MPG,Highway MPG,Drivetrain,Engine,Exterior Color,Interior Color,Transmission,Entertainment,Safety,Price
0,2012 Honda Civic EX,"151,000","Tacoma, WA 98409",3.0,Gasoline,28,39,FWD,1.8L I4 16V MPFI SOHC,Gray,Gray,5-Speed Automatic,['Bluetooth'],"['Brake Assist', 'Stability Control']","$6,900"
1,2016 Toyota Prius Two,"76,956","Tacoma, WA 98409",4.8,Hybrid,54,50,FWD,1.8L I4 16V MPFI DOHC Hybrid,Magnetic Gray Metallic,-1,Automatic CVT,['Bluetooth'],"['Backup Camera', 'Brake Assist', 'Stability C...","$15,000"
2,2020 Subaru Legacy Base,"7,633","Tacoma, WA 98409",4.2,Gasoline,27,35,AWD,2.5L H4 16V GDI DOHC,Crystal White Pearl,Gray,Automatic CVT,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...","$23,598"
3,2020 Mazda CX-5 Touring,"27,188","Tacoma, WA 98409",4.5,Gasoline,25,31,FWD,2.5L I4 16V GDI DOHC,Sonic Silver Metallic,Black,6-Speed Automatic,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...","$25,900"
4,2020 Hyundai Santa Fe SEL 2.4,"20,404","Tacoma, WA 98409",4.2,Gasoline,21,27,AWD,2.4L I4 16V GDI DOHC,Symphony Silver,Black,8-Speed Automatic,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...","$26,996"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10692,2016 Ford F-150,"86,836","Omak, WA 98841",4.8,Gasoline,-1,234,4WD,2.7L V6 24V GDI DOHC Twin Turbo,Caribou,-1,6-Speed Automatic,-1,"['Brake Assist', 'Stability Control']","$29,495"
10693,2001 Dodge Ram 1500,"179,561","Omak, WA 98841",4.8,Gasoline,13,17,4WD,5.2L V8 16V MPFI OHV,Patriot Blue Pearlcoat,-1,4-Speed Automatic,-1,-1,"$6,999"
10694,2016 Jeep Cherokee Limited,"127,717","Omak, WA 98841",4.8,Gasoline,21,28,AWD,2.4L I4 16V MPFI SOHC,Bright White Clearcoat,Black,9-Speed Automatic,['Bluetooth'],"['Backup Camera', 'Brake Assist', 'Stability C...","$15,495"
10695,2014 Ford F-150 FX2,"48,130","Washougal, WA 98671",4.8,E85 Flex Fuel,-1,-1,RWD,5.0L V8 32V MPFI DOHC,Sterling Gray Metallic,Steel Gray,6-Speed Automatic,-1,['Stability Control'],"$25,900"


In [3]:
df.sample(50)

,Name,Mileage,Address,Rating,Fuel Type,City MPG,Highway MPG,Drivetrain,Engine,Exterior Color,Interior Color,Transmission,Entertainment,Safety,Price
8588,2018 Audi S4 3.0T Premium Plus,"44,302","Fife, WA 98424",4.8,Gasoline,21,30,AWD,3.0L V6 24V GDI DOHC Turbo,Navarra Blue Metallic,Black,8-Speed Automatic,"['Bluetooth', 'Premium Sound System']","['Backup Camera', 'Brake Assist', 'Stability C...","$40,000"
6638,2018 Mazda CX-5 Touring,"43,454","Edmonds, WA 98026",4.2,Gasoline,24,30,AWD,2.5L I4 16V,Eternal Blue Mica,Black,6-Speed Automatic,['Bluetooth'],"['Backup Camera', 'Brake Assist', 'Stability C...","$22,402"
1222,Certified 2019 Honda CR-V EX-L,"26,799","Renton, WA 98057",4.6,Gasoline,27,33,AWD,1.5L I4 16V GDI DOHC Turbo,Black,-1,Automatic CVT,['Bluetooth'],"['Backup Camera', 'Brake Assist', 'Stability C...","$27,995"
524,2017 GMC Sierra 1500 Denali,"44,346","Auburn, WA 98002",4.7,Gasoline,15,21,4WD,6.2L V8 16V GDI OHV,White Frost Tri-Coat,Jet Black,8-Speed Automatic,"['Bluetooth', 'Premium Sound System']","['Backup Camera', 'Brake Assist', 'Stability C...","$44,250"
9527,2020 Toyota Tacoma TRD Off Road,"28,142","Seattle, WA 98104",1.0,Gasoline,18,22,4WD,3.5L V6 24V PDI DOHC,Magnetic Gray Metallic,-1,6-Speed Automatic,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...","$36,297"
2058,2017 Audi Q7 3.0T Premium Plus,"38,431","Seattle, WA 98101",3.6,Gasoline,-1,-1,AWD,Intercooled Supercharger Premium Unleaded V-6...,Gray,-1,Automatic,"['Bluetooth', 'Premium Sound System']","['Backup Camera', 'Brake Assist', 'Stability C...","$36,980"
6751,2018 BMW X5 eDrive xDrive40e,"26,870","Bellevue, WA 98004",4.6,Hybrid,-1,-1,AWD,2.0L I4 16V GDI DOHC Turbo Hybrid,Carbon Black Metallic,Black,8-Speed Automatic,['Bluetooth'],"['Backup Camera', 'Brake Assist', 'Stability C...","$40,888"
7729,2014 Toyota Sienna XLE,"64,775","Auburn, WA 98001",5.0,Gasoline,18,25,FWD,3.5L V6 24V MPFI DOHC,White,Gray,6-Speed Automatic,['Bluetooth'],"['Backup Camera', 'Brake Assist', 'Stability C...","$19,500"
1526,2016 Subaru Outback 2.5i Premium,"66,584","Renton, WA 98057",4.3,Gasoline,25,33,AWD,2.5L 4-Cylinder DOHC 16V,Twilight Blue Metallic,slate black,CVT Lineartronic,['Bluetooth'],"['Backup Camera', 'Brake Assist', 'Stability C...","$18,999"
1400,2006 Chrysler Pacifica Touring,"132,237","Tacoma, WA 98409",4.8,Gasoline,17,23,FWD,3.5L V6 24V MPFI SOHC,Brilliant Black Crystal Pearlcoat,Dark Slate,4-Speed Automatic,['Premium Sound System'],-1,"$7,490"


In [4]:
df[df.Name.str.contains('Certified')]

,Name,Mileage,Address,Rating,Fuel Type,City MPG,Highway MPG,Drivetrain,Engine,Exterior Color,Interior Color,Transmission,Entertainment,Safety,Price
7,Certified 2018 Mercedes-Benz GLA 250 Base 4MATIC,"37,225","Fife, WA 98424",4.4,E85 Flex Fuel,23,31,AWD,2.0L I4 16V GDI DOHC Turbo Flexible Fuel,Red,Shara Beige,7-Speed Automatic with Auto-Shift,"['Bluetooth', 'Premium Sound System']","['Backup Camera', 'Brake Assist', 'Stability C...","$28,726"
35,Certified 2017 RAM 1500 Big Horn,"37,322","Enumclaw, WA 98022",4.4,Gasoline,16,23,-1,5.7L V8 16V MPFI OHV,Brilliant Black Crystal Pearlcoat,Diesel Gray,8-Speed Automatic,['Bluetooth'],"['Brake Assist', 'Stability Control']","$37,455"
61,Certified 2018 BMW X1 xDrive28i,"30,098","Bellevue, WA 98005",4.6,Gasoline,22,31,AWD,2.0L I4 16V GDI DOHC Turbo,Glacier Silver Metallic,-1,8-Speed Automatic,['Bluetooth'],"['Backup Camera', 'Brake Assist', 'Stability C...","$28,560"
62,Certified 2018 Mercedes-Benz E 300,"24,886","Bellevue, WA 98005",4.6,Gasoline,21,29,AWD,2.0L I4 16V GDI DOHC Turbo,Obsidian Black Metallic,-1,9-Speed Automatic,"['Bluetooth', 'Premium Sound System']","['Brake Assist', 'Stability Control', 'Automat...","$44,450"
73,Certified 2018 Chevrolet Camaro 1LT,"40,792","Kirkland, WA 98033",4.7,Gasoline,22,31,RWD,2.0L I4 16V GDI DOHC Turbo,Silver Ice Metallic,Jet Black,8-Speed Automatic,"['Bluetooth', 'Premium Sound System', 'Apple C...","['Backup Camera', 'Brake Assist', 'Stability C...","$23,999"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10633,Certified 2018 Mercedes-Benz GLA 250 Base,"21,438","Lynnwood, WA 98037",4.6,Gasoline,24,33,AWD,Intercooled Turbo Premium Unleaded I-4 2.0 L/121,Polar Silver Metallic,-1,Automatic,-1,-1,"$29,995"
10636,Certified 2018 Mercedes-Benz C 300 4MATIC,"19,672","Lynnwood, WA 98037",4.6,Gasoline,23,32,AWD,Intercooled Turbo Premium Unleaded I-4 2.0 L/121,Iridium Silver Metallic,Crystal Grey/Black,Automatic,-1,-1,"$35,480"
10656,Certified 2020 Mercedes-Benz GLB 250 Base 4MATIC,"7,762","Fife, WA 98424",4.4,Gasoline,23,31,AWD,2.0L I4 16V GDI DOHC Turbo,White,-1,8-Speed Automatic with Auto-Shift,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...","$42,944"
10672,Certified 2020 Chevrolet Equinox Premier w/2LZ,"21,899","Enumclaw, WA 98022",4.3,Gasoline,22,28,AWD,2.0L I4 16V GDI DOHC Turbo,Nightfall Gray Metallic,Medium Ash Gray,9-Speed Automatic,"['Bluetooth', 'Premium Sound System', 'Apple C...","['Backup Camera', 'Brake Assist', 'Stability C...","$27,592"


In [5]:
df['Name'] = df['Name'].apply(lambda x: x.replace('Certified ','').strip())

In [6]:
df['Model Year'] = df['Name'].apply(lambda x: x.split(' ')[0])

In [7]:
df['Model Year'] = df['Model Year'].astype(int)
df['Model Year']

0        2012
1        2016
2        2020
3        2020
4        2020
         ... 
10692    2016
10693    2001
10694    2016
10695    2014
10696    2017
Name: Model Year, Length: 10697, dtype: int32

In [8]:
df.head()

,Name,Mileage,Address,Rating,Fuel Type,City MPG,Highway MPG,Drivetrain,Engine,Exterior Color,Interior Color,Transmission,Entertainment,Safety,Price,Model Year
0,2012 Honda Civic EX,"151,000","Tacoma, WA 98409",3.0,Gasoline,28,39,FWD,1.8L I4 16V MPFI SOHC,Gray,Gray,5-Speed Automatic,['Bluetooth'],"['Brake Assist', 'Stability Control']","$6,900",2012
1,2016 Toyota Prius Two,"76,956","Tacoma, WA 98409",4.8,Hybrid,54,50,FWD,1.8L I4 16V MPFI DOHC Hybrid,Magnetic Gray Metallic,-1,Automatic CVT,['Bluetooth'],"['Backup Camera', 'Brake Assist', 'Stability C...","$15,000",2016
2,2020 Subaru Legacy Base,"7,633","Tacoma, WA 98409",4.2,Gasoline,27,35,AWD,2.5L H4 16V GDI DOHC,Crystal White Pearl,Gray,Automatic CVT,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...","$23,598",2020
3,2020 Mazda CX-5 Touring,"27,188","Tacoma, WA 98409",4.5,Gasoline,25,31,FWD,2.5L I4 16V GDI DOHC,Sonic Silver Metallic,Black,6-Speed Automatic,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...","$25,900",2020
4,2020 Hyundai Santa Fe SEL 2.4,"20,404","Tacoma, WA 98409",4.2,Gasoline,21,27,AWD,2.4L I4 16V GDI DOHC,Symphony Silver,Black,8-Speed Automatic,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...","$26,996",2020


In [9]:
df['Mileage'] = df['Mileage'].apply(lambda x: x.replace(',','')).astype(int)

In [10]:
df['Price'] = df['Price'].apply(lambda x: x.replace('$','')).apply(lambda x: x.replace(',','')).astype(int)

In [11]:
df.head()

,Name,Mileage,Address,Rating,Fuel Type,City MPG,Highway MPG,Drivetrain,Engine,Exterior Color,Interior Color,Transmission,Entertainment,Safety,Price,Model Year
0,2012 Honda Civic EX,151000,"Tacoma, WA 98409",3.0,Gasoline,28,39,FWD,1.8L I4 16V MPFI SOHC,Gray,Gray,5-Speed Automatic,['Bluetooth'],"['Brake Assist', 'Stability Control']",6900,2012
1,2016 Toyota Prius Two,76956,"Tacoma, WA 98409",4.8,Hybrid,54,50,FWD,1.8L I4 16V MPFI DOHC Hybrid,Magnetic Gray Metallic,-1,Automatic CVT,['Bluetooth'],"['Backup Camera', 'Brake Assist', 'Stability C...",15000,2016
2,2020 Subaru Legacy Base,7633,"Tacoma, WA 98409",4.2,Gasoline,27,35,AWD,2.5L H4 16V GDI DOHC,Crystal White Pearl,Gray,Automatic CVT,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...",23598,2020
3,2020 Mazda CX-5 Touring,27188,"Tacoma, WA 98409",4.5,Gasoline,25,31,FWD,2.5L I4 16V GDI DOHC,Sonic Silver Metallic,Black,6-Speed Automatic,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...",25900,2020
4,2020 Hyundai Santa Fe SEL 2.4,20404,"Tacoma, WA 98409",4.2,Gasoline,21,27,AWD,2.4L I4 16V GDI DOHC,Symphony Silver,Black,8-Speed Automatic,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...",26996,2020


In [12]:
df['Fuel Type'].value_counts()

 Gasoline         9310
 Diesel            420
 Hybrid            420
 E85 Flex Fuel     368
 Electric          173
 Unknown             6
Name: Fuel Type, dtype: int64

In [13]:
df['City MPG'].median()

20.0

In [14]:
df.describe()

,Mileage,Rating,City MPG,Highway MPG,Price,Model Year
count,10697.000000,10697.000000,10697.000000,10697.000000,10697.000000,10697.000000
mean,44110.236702,4.269608,21.128915,28.163784,30474.719735,2016.757502
std,40150.680685,0.786603,15.155203,21.182875,17791.743255,4.075610
min,1.000000,1.000000,-1.000000,-1.000000,995.000000,1949.000000
25%,19785.000000,3.900000,17.000000,23.000000,19305.000000,2016.000000
50%,31205.000000,4.600000,20.000000,27.000000,27543.000000,2018.000000
75%,57411.000000,4.800000,24.000000,32.000000,37915.000000,2019.000000
max,385906.000000,5.000000,224.000000,384.000000,339880.000000,2021.000000


In [15]:
df['Engine'] = df['Engine'].apply(lambda x: x.strip().split(' ')[0][:3])


In [25]:
for i in range(len(df)):
    try:
        df['Engine'][i] = float(df['Engine'][i])
    except:
        df['Engine'][i] = np.nan

<ipython-input-25-7d8c61e125c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Engine'][i] = float(df['Engine'][i])
<ipython-input-25-7d8c61e125c1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Engine'][i] = np.nan


In [32]:
df.isna().sum()

Name                 0
Mileage              0
Address              0
Rating               0
Fuel Type            0
City MPG             0
Highway MPG          0
Drivetrain           0
Engine            1872
Exterior Color       0
Interior Color       0
Transmission         0
Entertainment        0
Safety               0
Price                0
Model Year           0
dtype: int64

In [33]:
df.head()

,Name,Mileage,Address,Rating,Fuel Type,City MPG,Highway MPG,Drivetrain,Engine,Exterior Color,Interior Color,Transmission,Entertainment,Safety,Price,Model Year
0,2012 Honda Civic EX,151000,"Tacoma, WA 98409",3.0,Gasoline,28,39,FWD,1.8,Gray,Gray,5-Speed Automatic,['Bluetooth'],"['Brake Assist', 'Stability Control']",6900,2012
1,2016 Toyota Prius Two,76956,"Tacoma, WA 98409",4.8,Hybrid,54,50,FWD,1.8,Magnetic Gray Metallic,-1,Automatic CVT,['Bluetooth'],"['Backup Camera', 'Brake Assist', 'Stability C...",15000,2016
2,2020 Subaru Legacy Base,7633,"Tacoma, WA 98409",4.2,Gasoline,27,35,AWD,2.5,Crystal White Pearl,Gray,Automatic CVT,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...",23598,2020
3,2020 Mazda CX-5 Touring,27188,"Tacoma, WA 98409",4.5,Gasoline,25,31,FWD,2.5,Sonic Silver Metallic,Black,6-Speed Automatic,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...",25900,2020
4,2020 Hyundai Santa Fe SEL 2.4,20404,"Tacoma, WA 98409",4.2,Gasoline,21,27,AWD,2.4,Symphony Silver,Black,8-Speed Automatic,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...",26996,2020


In [67]:
len(df['Safety'][3].split(','))

5

In [65]:
df['Num_entertain_fea'] = df['Entertainment'].apply(lambda x: len(x.split(',')))

In [66]:
df['Num_entertain_fea'].describe()

count    10697.000000
mean         1.731887
std          0.722220
min          1.000000
25%          1.000000
50%          2.000000
75%          2.000000
max          4.000000
Name: Num_entertain_fea, dtype: float64

In [68]:
df['Num_safe_fea'] = df['Safety'].apply(lambda x: len(x.split(',')))
df['Num_safe_fea'].describe()

count    10697.000000
mean         3.497523
std          1.521541
min          1.000000
25%          2.000000
50%          3.000000
75%          5.000000
max          7.000000
Name: Num_safe_fea, dtype: float64

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10697 entries, 0 to 10696
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               10697 non-null  object 
 1   Mileage            10697 non-null  int32  
 2   Address            10697 non-null  object 
 3   Rating             10697 non-null  float64
 4   Fuel Type          10697 non-null  object 
 5   City MPG           10697 non-null  int64  
 6   Highway MPG        10697 non-null  int64  
 7   Drivetrain         10697 non-null  object 
 8   Engine             8825 non-null   object 
 9   Exterior Color     10697 non-null  object 
 10  Interior Color     10697 non-null  object 
 11  Transmission       10697 non-null  object 
 12  Entertainment      10697 non-null  object 
 13  Safety             10697 non-null  object 
 14  Price              10697 non-null  int32  
 15  Model Year         10697 non-null  int32  
 16  Num_entertain_fea  106